# RAG pipeline 구축하기 4

챗봇만 있기보다는 검색창도 함께 있으면 좋을거 같다

라는 피드백을 받고 검색창에 검색어를 입력하면 게시물이 생성되게 해보자.


## 0. Setting 

.env 파일을 만들어 API키들을 넣어준다. 

나중에 Ollema를 사용해서 API 없이 local에서 작동가능한 LLM을 사용해 보자.



```bash
OPENAI_API_KEY='sk-proj-5m5haMMQ0Sgkctb7Udixxxxx'


```

In [3]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv
# API KEY 정보로드
load_dotenv()

True

## 1. RAG pipeline - 지금까지 필요한 라이브러리 다 불러오기

In [9]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import Chroma


# retriever
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.retrievers import EnsembleRetriever
from langchain_community.retrievers import BM25Retriever
import pickle

from langchain.chains import LLMChain, create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain.schema import HumanMessage

from utils.redis_utils import save_message_to_redis, get_messages_from_redis
from utils.prompt import *
from utils.config import config, metadata_field_info
from utils.update import convert_file_to_documents

## 지금까지 구현한 RAGPipeLine 클래스

In [7]:
class Ragpipeline:
    def __init__(self):
        # chatGPT API를 통해 llm 모델 로드
        self.llm = ChatOpenAI(
            model=config["llm_predictor"]["model_name"],  # chatgpt 모델 이름
            temperature=config["llm_predictor"]["temperature"],  # 창의성 0~1
        )
        
        # 초기화 리스트들 
        self.vector_store   = self.init_vectorDB()                  
        self.retriever      = self.init_retriever()                
        self.bm25_retriever = self.init_bm25_retriever()
        self.ensemble_retriever = self.init_ensemble_retriever()
        self.mq_ensemble_retriever = self.init_mq_ensemble_retriever()
        self.chain          = self.init_chat_chain()
        
        self.session_histories = {}
        self.current_user_email = None
        self.current_session_id = None

    def init_vectorDB(self, persist_directory=config["chroma"]["persist_dir"]):
        """vectorDB 설정"""
        embeddings = OpenAIEmbeddings(model=config["embed_model"]["model_name"])  # VectorDB에 저장될 데이터를 임베딩할 모델을 선언합니다.
        vector_store = Chroma(
            persist_directory=persist_directory,  # 기존에 vectordb가 있으면 해당 위치의 vectordb를 load하고 없으면 새로 생성합니다.
            embedding_function=embeddings,                      # 새롭게 데이터가 vectordb에 넣어질때 사용할 임베딩 방식을 정합니다, 저희는 위에서 선언한 embeddings를 사용합니다.
            collection_name = 'india',                          # india라는 이름을 정해줌으로써 나중에 vector store 관리 가능 
            collection_metadata = {'hnsw:space': 'cosine'},     # cosine 말고 l2 가 default / collection_metadata를 통해 유사도 검색에 사용될 공간('hnsw:space')을 'cosine'으로 지정하여, 코사인 유사도를 사용
        )
        return vector_store

    def init_retriever(self):            
        # base retriever 3 
        retriever = self.vector_store.as_retriever(
            search_type="mmr",                                              # mmr 검색 방법으로 
            search_kwargs={'fetch_k': 5, "k": 2, 'lambda_mult': 0.4},      # 상위 10개의 관련 context에서 최종 5개를 추리고 'lambda_mult'는 관련성과 다양성 사이의 균형을 조정하는 파라메타 default 값이 0.5
        )
        return retriever
    
    def init_bm25_retriever(self):
        all_docs = pickle.load(open(config["pkl_path"], 'rb'))
        bm25_retriever = BM25Retriever.from_documents(all_docs)
        bm25_retriever.k = 1                                            # BM25Retriever의 검색 결과 개수를 1로 설정합니다.
        return bm25_retriever
    
    def init_ensemble_retriever(self):
        ensemble_retriever = EnsembleRetriever(
            retrievers=[self.bm25_retriever, self.retriever],
            weights=[0.4, 0.6],
            search_type=config["ensemble_search_type"],  # mmr
        )
        return ensemble_retriever
    
    # 멀티쿼리 - 앙상블
    def init_mq_ensemble_retriever(self):
        mq_ensemble_retriever = MultiQueryRetriever.from_llm(
            llm=self.llm,
            retriever=self.ensemble_retriever
        )
        return mq_ensemble_retriever
    
    def init_chat_chain(self):
        # 1. 이어지는 대화가 되도록 대화기록과 체인
        history_aware_retriever = create_history_aware_retriever(self.llm, self.mq_ensemble_retriever, contextualize_q_prompt)      # self.mq_ensemble_retriever
        # 2. 문서들의 내용을 답변할 수 있도록 리트리버와 체인
        question_answer_chain = create_stuff_documents_chain(self.llm, qa_prompt)
        # 3. 1과 2를 합침
        rag_chat_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)
        
        return rag_chat_chain
    
    def chat_generation(self, question: str) -> dict:
        def get_session_history(session_id=None, user_email=None):
            session_id = session_id if session_id else self.current_session_id
            user_email = user_email if user_email else self.current_user_email

            if session_id not in self.session_histories:
                self.session_histories[session_id] = ChatMessageHistory()
                # Redis에서 세션 히스토리 불러오기
                history_messages = get_messages_from_redis(user_email, session_id)
                for message in history_messages:
                    self.session_histories[session_id].add_message(HumanMessage(content=message))
                    
            return self.session_histories[session_id]

        # 특정 유형의 작업(체인)에 메시지 기록을 추가, 대화형 애플리케이션 또는 복잡한 데이터 처리 작업을 구현할 때 이전 메시지의 맥락을 유지해야 할 필요가 있을 때 유용
        conversational_rag_chain = RunnableWithMessageHistory(      
            self.chain,                                 # 실행할 Runnable 객체
            get_session_history,                        # 세션 기록을 가져오는 함수
            input_messages_key="input",                 # 입력 메시지의 키
            history_messages_key="chat_history",        # 기록 메시지의 키
            output_messages_key="answer"                # 출력 메시지의 키 
        )
        response = conversational_rag_chain.invoke(
            {"input": question},
            config={"configurable": {"session_id": self.current_session_id}}            # 같은 session_id 를 입력하면 이전 대화 스레드의 내용을 가져오기 때문에 이어서 대화가 가능!
        )

        # Redis에 세션 히스토리 저장
        save_message_to_redis(self.current_user_email, self.current_session_id, question)
        save_message_to_redis(self.current_user_email, self.current_session_id, response["answer"])
        
        return response


In [8]:
pipeline = Ragpipeline()

### 다음과 같은 검색어를 입력하면 인도 통관에 대한 다양한 게시물이 사용자에게 보여지면 좋겠다.

In [1]:
question = '인도 통관'


아래와 같은 chain을 Ragpipeline 클래스에 넣자 

In [ ]:
# title_generator_prompt와 llm을 연결하고 리트리버랑 연결
def init_title_chain(self):
    question_answer_chain = create_stuff_documents_chain(self.llm, title_generator_prompt)
    rag_title_chain = create_retrieval_chain(self.retriever, question_answer_chain)
    return rag_title_chain

# post_generator_prompt와 llm을 연결하고 리트리버랑 연결 
def init_post_chain(self):
    question_answer_chain = create_stuff_documents_chain(self.llm, post_generator_prompt)
    rag_text_chain = create_retrieval_chain(self.mq_ensemble_retriever, question_answer_chain)
    return rag_text_chain

# 사용자 쿼리에 대한 게시물 제목 생성 
def title_generation(self, question: str):
    response = self.title_chain.invoke({'input': question})
    return response

# 생성된 게시물 제목에 대한 게시물 생성 
def post_generation(self, question: str):
    response = self.post_chain.invoke({'input': question})
    return response

In [26]:
class Ragpipeline:
    def __init__(self):
        # chatGPT API를 통해 llm 모델 로드
        self.llm = ChatOpenAI(
            model=config["llm_predictor"]["model_name"],  # chatgpt 모델 이름
            temperature=config["llm_predictor"]["temperature"],  # 창의성 0~1
        )
        
        # 초기화 리스트들 
        self.vector_store   = self.init_vectorDB()                  
        self.retriever      = self.init_retriever()                
        self.bm25_retriever = self.init_bm25_retriever()
        self.ensemble_retriever = self.init_ensemble_retriever()
        self.mq_ensemble_retriever = self.init_mq_ensemble_retriever()
        
        self.chain          = self.init_chat_chain()
        self.title_chain = self.init_title_chain()
        self.post_chain = self.init_post_chain()
        
        self.session_histories = {}
        self.current_user_email = None
        self.current_session_id = None

    def init_vectorDB(self, persist_directory=config["chroma"]["persist_dir"]):
        """vectorDB 설정"""
        embeddings = OpenAIEmbeddings(model=config["embed_model"]["model_name"])  
        vector_store = Chroma(
            persist_directory=persist_directory,  
            embedding_function=embeddings,                      
            collection_name = 'india',                          
            collection_metadata = {'hnsw:space': 'cosine'},     
        )
        return vector_store

# --1. 리트리버 ---------------------------------------------------------------------------------------------------------------------------------
    def init_retriever(self):            
        # base retriever 3 
        retriever = self.vector_store.as_retriever(
            search_type="mmr",                                              
            search_kwargs={'fetch_k': 5, "k": 2, 'lambda_mult': 0.4},     
        )
        return retriever
    
    def init_bm25_retriever(self):
        all_docs = pickle.load(open(config["pkl_path"], 'rb'))
        bm25_retriever = BM25Retriever.from_documents(all_docs)
        bm25_retriever.k = 1                                            
        return bm25_retriever
    
    def init_ensemble_retriever(self):
        ensemble_retriever = EnsembleRetriever(
            retrievers=[self.bm25_retriever, self.retriever],
            weights=[0.4, 0.6],
            search_type=config["ensemble_search_type"],  # mmr
        )
        return ensemble_retriever
    
    def init_mq_ensemble_retriever(self):
        mq_ensemble_retriever = MultiQueryRetriever.from_llm(
            llm=self.llm,
            retriever=self.ensemble_retriever
        )
        return mq_ensemble_retriever

# --2. 생성 chain 초기화 ---------------------------------------------------------------------------------------------------------------------------------
    def init_chat_chain(self):
        history_aware_retriever = create_history_aware_retriever(self.llm, self.mq_ensemble_retriever, contextualize_q_prompt)      # self.mq_ensemble_retriever

        question_answer_chain = create_stuff_documents_chain(self.llm, qa_prompt)

        rag_chat_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)
        return rag_chat_chain
    
    def init_title_chain(self):
        question_answer_chain = create_stuff_documents_chain(self.llm, title_generator_prompt)
        rag_title_chain = create_retrieval_chain(self.retriever, question_answer_chain)
        return rag_title_chain

    def init_post_chain(self):
        question_answer_chain = create_stuff_documents_chain(self.llm, post_generator_prompt)
        rag_text_chain = create_retrieval_chain(self.mq_ensemble_retriever, question_answer_chain)
        return rag_text_chain

# --3. 생성 결과 출력 ---------------------------------------------------------------------------------------------------------------------------------
    def chat_generation(self, question: str) -> dict:
        def get_session_history(session_id=None, user_email=None):
            session_id = session_id if session_id else self.current_session_id
            user_email = user_email if user_email else self.current_user_email

            if session_id not in self.session_histories:
                self.session_histories[session_id] = ChatMessageHistory()
                # Redis에서 세션 히스토리 불러오기
                history_messages = get_messages_from_redis(user_email, session_id)
                for message in history_messages:
                    self.session_histories[session_id].add_message(HumanMessage(content=message))
                
            return self.session_histories[session_id]

        conversational_rag_chain = RunnableWithMessageHistory(      
            self.chain,                                 # 실행할 Runnable 객체
            get_session_history,                        # 세션 기록을 가져오는 함수
            input_messages_key="input",                 # 입력 메시지의 키
            history_messages_key="chat_history",        # 기록 메시지의 키
            output_messages_key="answer"                # 출력 메시지의 키 
        )
        response = conversational_rag_chain.invoke(
            {"input": question},
            config={"configurable": {"session_id": self.current_session_id}}           
        )

        # Redis에 세션 히스토리 저장
        save_message_to_redis(self.current_user_email, self.current_session_id, question)
        save_message_to_redis(self.current_user_email, self.current_session_id, response["answer"])
        
        return response
    
    def title_generation(self, question: str):
        response = self.title_chain.invoke({'input': question})
        return response
    
    def post_generation(self, question: str):
        response = self.post_chain.invoke({'input': question})
        return response


In [27]:
pipeline = Ragpipeline()

## 게시물 생성

### 1. 게시물 제목 생성

In [28]:
question = '인도 통관'

In [29]:
titles = pipeline.title_generation(question)           # 제목 개수 정할 수 있음

titles

{'input': '인도 통관',
 'context': [Document(metadata={'category': '정책', 'page': 0, 'source': '[정책][제약산업정보포털][2019.04.08]인도 통관 및 운송.pdf', 'year': 2019}, page_content='5. 통관 및 운송\n \n \n가. 통관제도\n  \n \n통관 유형별 절차\n \n1) 정식통관 \n \n인도에서 일반적인 경우 통관에 소요되는 시간은 행정상 운송 수입의 경우 3~4 근무일, '),
  Document(metadata={'category': '정책', 'page': 1, 'source': '[정책][제약산업정보포털][2019.04.08]인도 통관 및 운송.pdf', 'year': 2019}, page_content='도 증빙이 충분치 않다는 이유로 통관을 거부하는 사례도 자주 발생한다.  \n \n  ㅇ 인도 지역별 세관관할: 인도의 세관 행정은 명목상 통일되어 있으나, 지역별 차이나 관할 세관 ')],
 'answer': '1. 인도 통관 절차: 성공적인 수입을 위한 가이드\n2. 인도 통관 시 주의사항: 지역별 세관 차이와 대응 전략\n3. 인도 통관 시간 단축 방법: 서류 준비와 절차 이해하기'}

In [31]:
print(titles['answer'])

1. 인도 통관 절차: 성공적인 수입을 위한 가이드
2. 인도 통관 시 주의사항: 지역별 세관 차이와 대응 전략
3. 인도 통관 시간 단축 방법: 서류 준비와 절차 이해하기


### 2. 생성한 게시물 제목을 바탕으로 게시물 생성하기

In [36]:
title = titles['answer'].split('\n')[0].split('.')[-1]

post = pipeline.post_generation(title)           # 제목 개수 정할 수 있음

post

{'input': ' 인도 통관 절차: 성공적인 수입을 위한 가이드',
 'context': [Document(metadata={'category': '정책', 'page': 0, 'source': '[정책][제약산업정보포털][2019.04.08]인도 통관 및 운송.pdf', 'year': 2019}, page_content='5. 통관 및 운송\n \n \n가. 통관제도\n  \n \n통관 유형별 절차\n \n1) 정식통관 \n \n인도에서 일반적인 경우 통관에 소요되는 시간은 행정상 운송 수입의 경우 3~4 근무일, '),
  Document(metadata={'category': '정책', 'page': 1, 'source': '[정책][제약산업정보포털][2019.04.08]인도 통관 및 운송.pdf', 'year': 2019}, page_content='도 증빙이 충분치 않다는 이유로 통관을 거부하는 사례도 자주 발생한다.  \n \n  ㅇ 인도 지역별 세관관할: 인도의 세관 행정은 명목상 통일되어 있으나, 지역별 차이나 관할 세관 '),
  Document(metadata={'source': '2023 인도진출전략.pdf', 'page_no': 31, 'category': '2. 경제 및 시장 분석', 'url': 'KOTRA', 'date': '2022.12.28'}, page_content='담당자를 통해 HS 코드를 확인하고 관련 증거 보관 및 품목분류 국제 분쟁 신고 또는 소송 진행 등 신중한 의사결정\n15\n2023 인도 진출전략\n22년 2월 연방예산안을 통해 점진적 관세 인상 계획 발표\n 발전소 공장 등의 건설에 사용되는 플랜트 수입물품에 대하여 기존 05 부과되던 관세율이 23년 9월 30일부터\n75로 인상 예정\n 단계별 제조업 지원 정책PMP Phased Manufacturing Programme 시행하여 웨어러블 디바이스 오디오 디바이스\n등 제조를 위한 수입물품에 대해 2022년부터 2026년까지 단계적 관세 인상 예정\n2

In [37]:
print(post['answer'])

## 인도 통관 절차: 성공적인 수입을 위한 가이드

인도 시장에 진출하려는 스타트업에게 통관 절차는 중요한 단계입니다. 이 가이드는 인도에서 성공적인 수입을 위해 알아야 할 주요 통관 절차와 관련 정보를 제공합니다. 

### 1. 정식 통관 절차

정식 통관은 인도에서 가장 일반적인 통관 유형입니다. 통관 절차는 다음과 같은 단계를 포함합니다:

- **서류 준비**: 수입자는 상업 송장, 포장 명세서, 원산지 증명서, 운송 서류 등 필요한 서류를 준비해야 합니다.
- **세관 신고**: 준비된 서류를 바탕으로 세관에 신고합니다. 이때 HS 코드를 정확히 기재하고, 관련 증거를 보관하는 것이 중요합니다.
- **세관 심사**: 세관은 제출된 서류를 검토하고, 필요시 추가 서류를 요청할 수 있습니다. 이 과정에서 서류가 충분하지 않으면 통관이 거부될 수 있습니다.
- **관세 납부**: 세관 심사가 완료되면 관세를 납부해야 합니다. 인도는 2023년 9월 30일부터 특정 품목에 대한 관세율을 인상할 예정이므로, 최신 관세율을 확인하는 것이 중요합니다.
- **물품 인도**: 관세 납부 후 물품을 인도받을 수 있습니다. 일반적으로 통관에 소요되는 시간은 3~4 근무일입니다.

### 2. 임시 통관 절차

임시 통관은 인도에 들여온 물품을 사용하지 않고 24개월 이내에 다시 반출할 경우 적용됩니다. 임시 통관 절차는 다음과 같습니다:

- **임시 수입 신고**: 임시 통관을 위해서는 임시 수입 신고서를 제출해야 합니다.
- **보증금 납부**: 물품의 가치를 보증하는 보증금을 납부해야 합니다. 이는 물품이 반출될 때 환불됩니다.
- **물품 사용 및 반출**: 물품을 사용하지 않고 24개월 이내에 다시 반출해야 합니다. 반출 시에는 원래 제출한 서류와 동일한 서류를 제출해야 합니다.

### 3. 비대면 통관 시스템: Turant Customs

인도는 2020년 10월 31일부터 비대면 통관 시스템인 Turant Customs를 시행하고 있습니다. 이 시스